In [69]:
import os
os.chdir('/home/jovyan/work')
import init
import reading
import do_calibration

import aavso
import numpy as np
import pandas as pd
from do_muniwin import write_munifind
from do_muniwin import getBestComparisonStars
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [70]:
import re

current_file = write_munifind(1.0, match_file)
comparison_stars = []
with open(current_file, 'r') as fp:
    for i, line in enumerate(fp):
        if i == 1:
            print(line)# 26th line
            m = re.search('Reference star:\s*(\d+),', line)            
            comparison_stars.append(int(m.group(1)))
            break
print(comparison_stars)

write munifind
quiet switch is -q 
Reference star: 3181, Aperture: 1

[3181]


In [71]:
ref_index = do_calibration.find_reference_frame_index()
print('reference index is', ref_index)
match_file = 'match????00.pht'
aperture_range = np.arange(0.2, 10, 0.2)
results = []
for aperture in aperture_range:
    print('aperture:', aperture)
    current_file = write_munifind(aperture, match_file)
    print(current_file)
    result = getBestComparisonStars(10000, current_file)
    results.append(result)

reference index is 2027
aperture: 0.2
write munifind
quiet switch is -q 
./inputfiles/WWCrA_allflat/munifind_0.2.txt
reading filename ./inputfiles/WWCrA_allflat/munifind_0.2.txt


FileNotFoundError: File b'./inputfiles/WWCrA_allflat/munifind_0.2.txt' does not exist

In [46]:
first_star = results[0]['STAR'].iloc[0]
second_star = results[0]['STAR'].iloc[1]
print("First star: {}, second star: {}".format(first_star, second_star))
diff_result_array = []
for df in results:
    first_stdev = df.loc[df['STAR'] == first_star]['MEAN_MAG']
    second_stdev = df.loc[df['STAR'] == second_star]['MEAN_MAG']
    #print("first: {},\n second: {}, {}".format(first_stdev, second_stdev, type(first_stdev)))
    diff_result = second_stdev.values - first_stdev.values
    sum_stdev = df['STDEV'].mean()
    #print(diff_result)
    #if(len(diff_result) == 1):
    diff_result_array = np.append(diff_result_array, sum_stdev)
print(diff_result_array)
%matplotlib inline
pd.Series(diff_result_array).plot()
print('Optimal aperture:',np.arange(0.2, 10, 0.2)[pd.Series(diff_result_array).idxmin()])


IndexError: list index out of range